In [ ]:
## Scraping using serp api and saving that pdf file
import requests
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

API_KEY = "API_KEY"  # replace with your SerpAPI key

def get_jobs(query, location="India"):
    url = "https://serpapi.com/search.json"
    params = {
        "engine": "google_jobs",
        "q": query,
        "location": location,
        "api_key": API_KEY
    }
    response = requests.get(url, params=params)
    data = response.json()

    jobs = []
    for job in data.get("jobs_results", []):
        jobs.append({
            "title": job.get("title"),
            "company": job.get("company_name"),
            "location": job.get("location"),
            "via": job.get("via"),
            "description": job.get("description"),
            "link": job.get("link"),
            "salary": job.get("detected_extensions", {}).get("salary", "Not provided")
        })
    return jobs

# Fetch jobs
roles = ["Data Analyst", "Data Scientist", "AI Engineer"]
all_jobs = []
for role in roles:
    all_jobs.extend(get_jobs(role))

# Save directly to PDF
pdf_file = "jobs_list.pdf"
c = canvas.Canvas(pdf_file, pagesize=letter)
width, height = letter

y = height - 50
for i, job in enumerate(all_jobs, start=1):
    text = f"{i}. {job['title']} - {job['company']} ({job['location']})"
    c.drawString(50, y, text)
    y -= 15
    c.drawString(60, y, f"Salary: {job['salary']}")
    y -= 15
    c.drawString(60, y, f"Via: {job['via']}")
    y -= 15
    c.drawString(60, y, f"Description: {job['description'][:100]}...")
    y -= 15
    c.drawString(60, y, f"Link: {job['link']}")
    y -= 30  # Space between jobs

    # If page is full, start a new page
    if y < 50:
        c.showPage()
        y = height - 50

c.save()
print(f"Jobs saved to {pdf_file}")
